In [69]:
import tensorflow as tf
import numpy as np
import os

def enqueue_patterns(path_to_data_file, 
                     batch_size, 
                     inp_size, 
                     targ_size, 
                     shuffle = False, 
                     buffer = None, 
                     seed = None,
                     allow_smaller_final_batch = True):
    
    filename_queue = tf.train.string_input_producer(string_tensor = [path_to_data_file], shuffle = False)
    reader = tf.TextLineReader(skip_header_lines=True, name='csv_reader')
    key, record_string = reader.read(filename_queue)
    defaults = [[default_val] for x in range(inp_size + targ_size)]
    defaults.insert(0,[''])
    examples = tf.decode_csv(record_string, record_defaults=defaults)
    p = tf.transpose(examples.pop(0))
    x = tf.transpose(tf.stack(examples[0:inp_size]))
    t = tf.transpose(tf.stack(examples[inp_size:inp_size + targ_size]))
    if shuffle:
        assert type(buffer) is int, 'provide buffer size if you want to shuffle examples'
        min_after_dequeue = buffer
        capacity = min_after_dequeue + 3 * batch_size
        p_batch, x_batch, t_batch = tf.train.shuffle_batch(
            tensors = [p, x, t], 
            batch_size = batch_size, 
            capacity = min_after_dequeue + 3 * batch_size,
            min_after_dequeue = min_after_dequeue,
            seed = seed,
        )
    else:
        p_batch, x_batch, t_batch = tf.train.batch(
            tensors = [p, x, t],
            batch_size = batch_size,
            num_threads = 1,
            capacity = 3 * batch_size,
        )

    return p_batch, x_batch, t_batch

def train(sess, examples, data_len, allow_smaller_final_batch=True):
    stdout = 'Label: \n{}\nInput: \n{}\nTarget: \n{}\n'
    print('>>> TRAIN <<<\n===================')
    for i in range(data_len // batch_size):
        print('step:',i)
        pat_name, inp, targ = sess.run([example for example in examples])
        print(stdout.format(pat_name, inp, targ))

def test(sess, examples, data_len):
    print('>>> TEST <<<\n===================')
    pat_lab, inp, targ = sess.run([example for example in examples])
    for pl, x, t in zip(pat_lab, inp, targ):
        print(pl, x, t)


In [72]:
batch_size = 2
inp_size = 2
targ_size = 1
default_val = 0.0
path='../train_data_{}.txt'
num_epochs = 10
data_len = 4

examples_train = enqueue_patterns(path.format('B'), batch_size, inp_size, targ_size, shuffle=True, buffer=100, seed=1, allow_smaller_final_batch=True)
examples_test = enqueue_patterns(path.format('C'), data_len, inp_size, targ_size, shuffle=False)


with tf.Session() as sess:
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
  # Start populating the filename queue.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for i in range(num_epochs):
            print('EPOCH:', i)
            if i==0 or i % 5 ==0:
                test(sess, examples_test, data_len)
            train(sess, examples_train, data_len, allow_smaller_final_batch=True)
           

    coord.request_stop()
    coord.join(threads)

EPOCH: 0
>>> TEST <<<
b'p00' [ 2.  2.] [ 2.]
b'p01' [ 3.  3.] [ 3.]
b'p10' [ 4.  4.] [ 4.]
b'p11' [ 5.  5.] [ 5.]
>>> TRAIN <<<
step: 0
Label: 
[b'p01' b'p11']
Input: 
[[ 0.  1.]
 [ 1.  1.]]
Target: 
[[ 1.]
 [ 0.]]

step: 1
Label: 
[b'p00' b'p00']
Input: 
[[ 0.  0.]
 [ 0.  0.]]
Target: 
[[ 0.]
 [ 0.]]

EPOCH: 1
>>> TRAIN <<<
step: 0
Label: 
[b'p11' b'p11']
Input: 
[[ 1.  1.]
 [ 1.  1.]]
Target: 
[[ 0.]
 [ 0.]]

step: 1
Label: 
[b'p01' b'p01']
Input: 
[[ 0.  1.]
 [ 0.  1.]]
Target: 
[[ 1.]
 [ 1.]]

EPOCH: 2
>>> TRAIN <<<
step: 0
Label: 
[b'p11' b'p10']
Input: 
[[ 1.  1.]
 [ 1.  0.]]
Target: 
[[ 0.]
 [ 1.]]

step: 1
Label: 
[b'p00' b'p00']
Input: 
[[ 0.  0.]
 [ 0.  0.]]
Target: 
[[ 0.]
 [ 0.]]

EPOCH: 3
>>> TRAIN <<<
step: 0
Label: 
[b'p01' b'p00']
Input: 
[[ 0.  1.]
 [ 0.  0.]]
Target: 
[[ 1.]
 [ 0.]]

step: 1
Label: 
[b'p10' b'p11']
Input: 
[[ 1.  0.]
 [ 1.  1.]]
Target: 
[[ 1.]
 [ 0.]]

EPOCH: 4
>>> TRAIN <<<
step: 0
Label: 
[b'p11' b'p01']
Input: 
[[ 1.  1.]
 [ 0.  1.]]
Target: 
[[ 0

In [4]:
a = np.array([[1,2,3],[4,5,6]])
aa = np.array([[11,22,33],[44,55,66]])
aaa = np.array([[111,222,333],[444,555,666]])
for i,ii,iii in zip(a,aa,aaa):
    print(i,ii,iii)

[1 2 3] [11 22 33] [111 222 333]
[4 5 6] [44 55 66] [444 555 666]


In [32]:
4//3

1